In [1]:
from label_studio_sdk import Client
from dotenv import load_dotenv
import os
import json

In [2]:
load_dotenv()

LS_URL = "http://localhost:8080"  
API_KEY = os.environ.get("API_KEY")

In [3]:
from label_studio_sdk.client import LabelStudio

ls = LabelStudio(
    base_url=LS_URL,  
    api_key=API_KEY,
)

In [4]:
label_config = """
<View>
  <Text name="input" value="$prompt"/>
  <Text name="output" value="$response"/>
  <Choices name="reward" toName="output">
    <Choice value="1">Good</Choice>
    <Choice value="0">Bad</Choice>
  </Choices>
</View>
"""

In [5]:
proj = ls.projects.create(
    title="RLVR Scoring",
    description="Annotate responses for RLVR training.",
    label_config=label_config,  
    color="#FF8800"
)
pid = proj.id
print(f"Created project {pid}")


verify:True
Created project 53


In [6]:
tasks = [
    {
        "prompt": "Solve for x: 2x + 3 = 7",
        "response": "Subtract 3 from both sides: 2x = 4, then divide by 2. So, x = 2."
    },
    {
        "prompt": "What is the derivative of x^2?",
        "response": "2x"
    },
    {
        "prompt": "Calculate the area of a circle with radius 3",
        "response": "To calculate the area of a circle, use the formula Area = π * r^2. With r = 3, Area = π * 3 * 3 = 9π."
    }
]

ls.projects.import_tasks(id=pid, request=tasks)
print("Uploaded tasks")


Uploaded tasks


(Human annotation happens in UI)

In [7]:
tasks = ls.tasks.list(project=proj.id)

dataset = []
for task in tasks:
    obj = task.model_dump()
    item = {
        "prompt": obj["data"].get("prompt"),
        "response": obj["data"].get("response"),
        "reward": None   
    }
    
    for ann in obj.get("annotations", []):
        for r in ann.get("result", []):
            if "value" in r and "choices" in r["value"]:
                choice = r["value"]["choices"][0]
                if choice == "Good":
                    item["reward"] = 1
                elif choice == "Bad":
                    item["reward"] = 0
    dataset.append(item)


with open("rlvr_data.json", "w") as f:
    json.dump(dataset, f, indent=2)

print(f"Exported {len(dataset)} tasks in RLVR format (prompt, response, reward)")

Exported 3 tasks in RLVR format (prompt, response, reward)
